## 自动求导
假设对函数$y=2x^Tx$,关于列向量x求导

In [1]:
import torch 

In [2]:
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [4]:
x.requires_grad_(True) #等价于'x = torch.arange(4.0,requires_grad=True)
x.grad #默认值是None

In [5]:
y = 2 * torch.dot(x,x)  #输入元素只允许是一维的tensor,torch.dot对应元素相乘再相加
y   ## 相当于是y = 2*(x1*x1+x2*x2+x3*x3+x4*x4)

tensor(28., grad_fn=<MulBackward0>)

通过调用反向传播函数来自动计算y关于x每个分量的梯度

In [6]:
y.backward()
x.grad        ## [4*x1,4*x2,4*x3,4*x4]

tensor([ 0.,  4.,  8., 12.])

In [7]:
x.grad == 4 * x

tensor([1, 1, 1, 1], dtype=torch.uint8)

现在计算$x$的另一个函数。

In [8]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()    # 把x所有的梯度清零
y = x.sum()      # y = x1+x2+x3+x4
y.backward()
x.grad

tensor([1., 1., 1., 1.])

深度学习中，我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和。
假设y不是一个标量，是一个向量，那么它的backward function是一个矩阵,相当于一个向量对一个向量求导，是一个矩阵， 
> https://zhuanlan.zhihu.com/p/263777564

实际上机器学习很少对向量的函数求导，都是把向量元素相加变为标量后再求导

In [11]:
# 对非标量调用backward需要传入一个gradient参数，
x.grad.zero_()
y = x * x  #这里是哈达玛积
#等价于 y.backward(torch.ones(len(x)))  #和我之前看的那个csdn一样
y.sum().backward()  #变为标量
x.grad

tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外  **detach**

In [13]:
x.grad.zero_()
y = x * x
u = y.detach()  ## 把y当作一个常数而不是一个关于x的函数，赋给u
z = u * x

z.sum().backward()
x.grad == u   ## 对x求导，梯度就是u


tensor([1, 1, 1, 1], dtype=torch.uint8)

In [14]:
x.grad.zero_()
y.sum().backward()  #y还是x的函数
x.grad == 2 * x

tensor([1, 1, 1, 1], dtype=torch.uint8)

即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度

In [15]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c
a = torch.randn(size=(),requires_grad=True) ##size = () size为空的话，就是一个标量
d = f(a)
d.backward()

a.grad == d/a  ##函数f之改变了a前面的系数

tensor(1, dtype=torch.uint8)